# Import GAMS and GAMS Transfer

In [70]:
%reload_ext gams.magic

In [71]:
import pandas as pd
import gams.transfer as gt

# Preprocess Data in Python

In [60]:
class Product:
    def __init__(self, name, price, volume, fat, category,
                 muscle=0, lung=0, liver=0, kidney=0, spleen=0, heart=0, 
                 fish=0, pansen=0, rfk=0, vegetable=0, fruit=0, inventory=0):
        self.name = name
        self.price = price
        self.volume = volume
        self.fat = fat
        self.category = category
        self.inventory = inventory
        self.muscle = muscle
        self.fish = fish
        self.liver = liver
        self.lung = lung
        self.kidney = kidney
        self.spleen = spleen
        self.heart = heart
        self.pansen = pansen
        self.rfk = rfk
        self.vegetable = vegetable
        self.fruit = fruit
        self.price_per_kg = self.calc_price_per_kg()
        self.composition = {
            'Muscle': muscle + fish,
            'Intestinals': liver + lung + kidney + spleen + heart,
            'Pansen': pansen,
            'RFK': rfk,
            'Vegetable': vegetable,
            'Fruit': fruit
        }
        
    def calc_price_per_kg(self):
        return round((self.price / self.volume) * 1000, 2)

In [61]:
def check_components(df):
    assert df['Share'].sum() == 2, 'Components do not add up to 100%'

In [62]:
weeks = 4
weight = 35 # dogs weight
perc =  0.02 # percent of body weight 
activity = 1 # dog activity level 
fatlb = 0.11 # lower bound of fat
fatub = 0.15 # upper bound of fat

# RFK 20% bei nur weichen RFK
# RFK 15% bei gemischten RFK
component_share = pd.DataFrame([
    ['Muscle', 0.45],
    ['Intestinals', 0.15],
    ['Pansen', 0.2],
    ['RFK', 0.2],
    ['Vegetable', 0.75],
    ['Fruit', 0.25]
], columns=['Component', 'Share'])

check_components(component_share)

# component_share = pd.DataFrame([
#     ['Muscle', 0.7],
#     ['Intestinals', 0.15],
#     ['Pansen', 0],
#     ['RFK', 0.15],
#     ['Vegetable', 0.75],
#     ['Fruit', 0.25]
# ], columns=['Component', 'Share'])

intestinal_share = pd.DataFrame([
    ['Liver', 2/5],
    ['Heart', 1/5],
    ['Kidney', 1/5],
    ['Spleen', 1/5]
], columns=['Intestinal', 'Share'])

modules_share = {
    'Meat': 0.8,
    'Veg & Fruit': 0.2
}

In [63]:
products = [
    # Pferd
    Product(name='Pferdefleisch', category='Horse', volume=500, price=3.58, muscle=1, fat=0.23),
    Product(name='PferdefleischwuerfelFettig', category='Horse', volume=2000, price=12.88, muscle=1, fat=0.254),
    Product(name='PferdefleischKlumpen', category='Horse', volume=2000, price=16.98, muscle=1, fat=0.05),
    Product(name='Pferdeherz', category='Horse', volume=500, price=4.98, heart=1, fat=0.12),
    Product(name='Pferdemilz', category='Horse', volume=500, price=5.48, spleen=1, fat=0.051),
    Product(name='Pferdeniere', category='Horse', volume=500, price=3.98, kidney=1, fat=0.085),
    Product(name='Pferdeleber', category='Horse', volume=500, price=3.98, liver=1, fat=0.04),
    Product(name='Pferdefett', category='Horse', volume=500, price=3.48, muscle=1, fat=0.91),
    # Rind
    Product(name='Kalbsfleischabschnitte', category='Beef', volume=500, price=3.08, muscle=1, fat=0.35),
    Product(name='RindfleischKlumpen', category='Beef', volume=2000, price=12.68, muscle=1, fat=0.12),
    Product(name='Rindermuskelfleischwuerfel', category='Beef', volume=500, price=3.58, muscle=1, fat=0.09),
    Product(name='Rinderfett', category='Beef', volume=500, price=2.98, muscle=1, fat=0.91),
    Product(name='Rinderherzwuerfel', category='Beef', volume=500, price=3.08, heart=1, fat=0.05),
    Product(name='RinderniereAmStueck', category='Beef', volume=500, price=2.28, kidney=1, fat=0.085),
    Product(name='RindermilzWuerfel', category='Beef', volume=500, price=2.18, spleen=1, fat=0.051),
    Product(name='RinderLeberNierenMilzMix', category='Beef', volume=500, price=2.78, liver=0.5, kidney=0.25, spleen=0.25, fat=0.039),
    # Lamm
    Product(name='Lammbauchlappen', category='Lamb', volume=1000, price=10.48, muscle=1, fat=0.3),
    Product(name='Lammfleisch', category='Lamb', volume=500, price=4.38, muscle=1, fat=0.048),
    Product(name='Lammfleischabschnitte', category='Lamb', volume=500, price=5.08, muscle=1, fat=0.15),
    Product(name='Lammfett', category='Lamb', volume=500, price=3.98, muscle=1, fat=0.91),
    Product(name='Lammpansen', category='Lamb', volume=1000, price=5.38, pansen=1, fat=0.14),
    Product(name='LammleberStuecke', category='Lamb', volume=500, price=3.28, liver=1, fat=0.05),
    Product(name='LammniereStuecke', category='Lamb', volume=500, price=3.78, kidney=1, fat=0.06),
    Product(name='LammmilzStuecke', category='Lamb', volume=500, price=4.68, spleen=1, fat=0.027),
    Product(name='LamminnereienMix', category='Lamb', volume=250, price=2.08, heart=0.4, kidney=0.3, liver=0.3, fat=0.07),
    Product(name='LammherzStuecke', category='Lamb', volume=500, price=4.18, heart=1, fat=0.05),
    # Geflügel
    Product(name='Haehnchenherz', category='Chicken', volume=500, price=3.08, heart=1, fat=0.05),
    Product(name='Huehnerfett', category='Chicken',  volume=1000, price=3.48, muscle=1, fat=0.995),
    Product(name='Huehnerfleisch', category='Chicken', volume=1000, price=7.88, muscle=1, fat=0.142),
    Product(name='Gefluegelinnereien-Mix', category='Chicken', volume=500, price=2.98, heart=0.5, muscle=0.5, fat=0.05),
    # Fish
    Product(name='Lachs', category='Fish', volume=500, price=2.98, fish=1, fat=0.075),
    # Knochen
    Product(name='Kaninchenkarkasse', category='Rabbit', volume=1000, price=5.18, rfk=1, fat=0.15),
    Product(name='Rinderbrustbein', category='Beef', volume=1000, price=3.68, rfk=1, fat=0.14),
    Product(name='Lammrippchen', category='Lamb', volume=1000, price=4.99, rfk=1, fat=0.15),
    Product(name='Huehnerhaelse', category='Chicken', volume=1000, price=3.28, rfk=1, fat=0.22),
    # Vegetable & Fruit
    Product(name='Vegetables', category='Vegetables', volume=1, price=0, vegetable=1, fat=0),
    Product(name='Fruit', category='Fruits', volume=1, price=0, fruit=1, fat=0)
] 

data = {
    'Name': [p.name for p in products], 'Category': [p.category for p in products], 
    'Inventory': [p.inventory for p in products], 
    'Price [€/kg]': [p.price_per_kg for p in products], 'Price': [p.price for p in products], 
    'Volume': [p.volume for p in products], 'Fat': [p.fat for p in products] 
}
df = pd.DataFrame(data)

cont_comp = {'Name': [], 'Component': [], 'Share': []}
for c in component_share['Component']:
    for p in products:
        cont_comp['Name'].append(p.name)
        cont_comp['Component'].append(c)
        cont_comp['Share'].append(p.composition[c])
cont_comp_df = pd.DataFrame(cont_comp)

int_comp = {'Name': [], 'Intestinal': [], 'Share': []}
for p in products:
    int_comp['Name'].append(p.name)
    int_comp['Intestinal'].append('Liver')
    int_comp['Share'].append(p.liver)
    #
    int_comp['Name'].append(p.name)
    int_comp['Intestinal'].append('Heart')
    int_comp['Share'].append(p.heart)
    #
    int_comp['Name'].append(p.name)
    int_comp['Intestinal'].append('Kidney')
    int_comp['Share'].append(p.kidney)
    #
    int_comp['Name'].append(p.name)
    int_comp['Intestinal'].append('Spleen')
    int_comp['Share'].append(p.spleen)
int_comp_df = pd.DataFrame(int_comp)

In [64]:
# inventory
def add_inventory(df, product, inventory):
    assert df['Name'].isin([product]).any(), f'{product} not in DataFrame' 
    df.loc[df['Name'] == product, 'Inventory'] = inventory

add_inventory(df, 'LammleberStuecke', 260 + 500 + 80)
add_inventory(df, 'Lammfleisch', 1050)
add_inventory(df, 'LammherzStuecke', 250)
add_inventory(df, 'Lammpansen', 400)

add_inventory(df, 'Pferdeherz', 630)

add_inventory(df, 'Kalbsfleischabschnitte', 150)
add_inventory(df, 'Rinderfett', 200)
add_inventory(df, 'RindfleischKlumpen', 2000)
add_inventory(df, 'Rinderherzwuerfel', 250)
add_inventory(df, 'RindermilzWuerfel', 250)

add_inventory(df, 'Huehnerhaelse', 500)
add_inventory(df, 'Gefluegelinnereien-Mix', 1500)

add_inventory(df, 'Kaninchenkarkasse', 700)
add_inventory(df, 'Lammrippchen', 500)

add_inventory(df, 'Lachs', 4000)

df.sort_values(by='Price [€/kg]')

,Name,Category,Inventory,Price [€/kg],Price,Volume,Fat
36,Fruit,Fruits,0,0.00,0.00,1,0.000
35,Vegetables,Vegetables,0,0.00,0.00,1,0.000
34,Huehnerhaelse,Chicken,500,3.28,3.28,1000,0.220
27,Huehnerfett,Chicken,0,3.48,3.48,1000,0.995
32,Rinderbrustbein,Beef,0,3.68,3.68,1000,0.140
14,RindermilzWuerfel,Beef,250,4.36,2.18,500,0.051
13,RinderniereAmStueck,Beef,0,4.56,2.28,500,0.085
33,Lammrippchen,Lamb,500,4.99,4.99,1000,0.150
31,Kaninchenkarkasse,Rabbit,700,5.18,5.18,1000,0.150
20,Lammpansen,Lamb,400,5.38,5.38,1000,0.140


In [65]:
daylydemand = weight * perc * activity * 1000;
weeklydemand = daylydemand * 7;

demand = [
    ['Muscle', component_share[component_share['Component'] == 'Muscle']['Share'].values[0] * modules_share['Meat'] * weeklydemand * weeks],
    ['Intestinals', component_share[component_share['Component'] == 'Intestinals']['Share'].values[0] * modules_share['Meat'] * weeklydemand * weeks],
    ['Pansen', component_share[component_share['Component'] == 'Pansen']['Share'].values[0] * modules_share['Meat'] * weeklydemand * weeks],
    ['RFK', component_share[component_share['Component'] == 'RFK']['Share'].values[0] * modules_share['Meat'] * weeklydemand * weeks],
    ['Vegetable', component_share[component_share['Component'] == 'Vegetable']['Share'].values[0] * modules_share['Veg & Fruit'] * weeklydemand * weeks],
    ['Fruit', component_share[component_share['Component'] == 'Fruit']['Share'].values[0] * modules_share['Veg & Fruit'] * weeklydemand * weeks],
]
    
demand_df = pd.DataFrame(demand)

demand_df

,0,1
0,Muscle,7056.0
1,Intestinals,2352.0
2,Pansen,3136.0
3,RFK,3136.0
4,Vegetable,2940.0
5,Fruit,980.0


# Define Variable Bounds

In [66]:
z_bounds = pd.DataFrame({
    'Name': [p.name for p in products], 'upper': [float('inf') for p in products]
})
z_bounds.loc[z_bounds['Name'] == 'Lammrippchen', 'upper'] = 0
z_bounds.loc[z_bounds['Name'] == 'Rinderbrustbein', 'upper'] = 0

In [67]:
df[['Name', 'Inventory']]

,Name,Inventory
0,Pferdefleisch,0
1,PferdefleischwuerfelFettig,0
2,PferdefleischKlumpen,0
3,Pferdeherz,630
4,Pferdemilz,0
5,Pferdeniere,0
6,Pferdeleber,0
7,Pferdefett,0
8,Kalbsfleischabschnitte,150
9,RindfleischKlumpen,2000


# Pass Data to GAMS

In [72]:
%gams_reset
m = gams.exchange_container

# create sets
p = m.addSet('p', records=df['Name'], description='products')
c = m.addSet('c', records=component_share['Component'], description='components')
i = m.addSet('i', records=intestinal_share['Intestinal'], description='intestinals')
INT = m.addSet('INT', c, records=['Intestinals'], description='identifies the intestinal component')
FISH = m.addSet('FISH', p, records=df[df['Category'] == 'Fish'][['Name', 'Category']])
BEEF = m.addSet('BEEF', p, records=df[df['Category'] == 'Beef'][['Name', 'Category']])
LAMB = m.addSet('LAMB', p, records=df[df['Category'] == 'Lamb'][['Name', 'Category']])
CHICKEN = m.addSet('CHICKEN', p, records=df[df['Category'] == 'Chicken'][['Name', 'Category']])

# create parameters
price = m.addParameter('price', p, records=df[['Name', 'Price']], description="price of product p")

volume = m.addParameter('volume', p, records=df[['Name', 'Volume']], 
                        description='volume of product p')

inventory = m.addParameter('inventory', p, records=df[['Name', 'Inventory']], 
                           description='inventory of product p')

fat = m.addParameter('fat', p, records=df[['Name', 'Fat']], description='fat share of product p')

demand = m.addParameter('demand', c, records=demand_df, description='demand for each component')

content_components = m.addParameter('content_components', [p,c], records=cont_comp_df, 
                                    description='components of product p')

intestinal_components = m.addParameter('intestinal_components', [p,i], records=int_comp_df,
                                       description='intestinal components of product p')

int_share = m.addParameter('intestinal_share', i, records=intestinal_share,
                           description='intestinal share')

# create scalars
m.addParameter('weeks', records=weeks, description="weeks to pack")
m.addParameter('weeklydemand', records=weeklydemand, description="weekly demand")
m.addParameter('weight', records=weight, description="dogs desired body weight")
m.addParameter('perc', records=perc, description="percent of body weight")
m.addParameter('activity', records=activity, description="dog activity level")
m.addParameter('fatlb', records=fatlb, description="lower bound on fat")
m.addParameter('fatub', records=fatub, description="upper bound on fat")

# create variables
F = m.addVariable('F', description='objective function value')
X = m.addVariable('X', 'positive', p)
Y = m.addVariable('Y', 'integer', p)
Z = m.addVariable('Z', 'integer', p, records=z_bounds)

# write the GDX
m.write("data.gdx")

# Run GAMS

In [73]:
%%gams
Equations
objective   Objective Function
total       Total
totalUB     Total
components
intestinal_constraints
fishday
fat_UB
fat_LB
buy
rounding
bbb
;

objective.. f =e= sum(p, price(p) * Z(p));
total.. sum(p, X(p)) =g= 0.98 * weeklydemand * weeks;
totalUB.. sum(p, X(p)) =l= 1.02 * weeklydemand * weeks;
components(c).. sum(p, content_components(p,c) * X(p)) =g= demand(c) - 26;
intestinal_constraints(i).. sum(p, intestinal_components(p,i) * X(p)) =g= intestinal_share(i) * sum(c$INT(c), demand(c));
fishday.. sum(FISH(p), X(p)) =e= 500 * weeks;
fat_LB.. sum(p, fat(p) * X(p)) =g= fatlb * sum(p, X(p));
fat_UB.. sum(p, fat(p) * X(p)) =l= fatub * sum(p, X(p)); 
buy(p).. inventory(p) + volume(p) * Z(p) =g= X(p);
rounding(p).. X(p) =e= 50 * Y(p);
bbb.. X('Lammrippchen') + X('Rinderbrustbein') =l= 0.5 * demand('RFK');

model BARF /all/;

solve BARF minimizing f using MIP;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),33.42,91,112,MIP,CPLEX,0.016


In [75]:
%gams_lst -e

# Review Results

In [76]:
X.records.loc[X.records['level'] > 1][['p_0', 'level']]

,p_0,level
8,Kalbsfleischabschnitte,150.0
9,RindfleischKlumpen,1900.0
11,Rinderfett,200.0
13,RinderniereAmStueck,350.0
14,RindermilzWuerfel,350.0
15,RinderLeberNierenMilzMix,500.0
17,Lammfleisch,1050.0
20,Lammpansen,3150.0
21,LammleberStuecke,700.0
27,Huehnerfett,1000.0


In [77]:
Z.records.loc[Z.records['level'] > 0][['p_0', 'level']]

,p_0,level
13,RinderniereAmStueck,1.0
14,RindermilzWuerfel,1.0
15,RinderLeberNierenMilzMix,1.0
20,Lammpansen,3.0
27,Huehnerfett,1.0
34,Huehnerhaelse,2.0
35,Vegetables,2950.0
36,Fruit,1000.0


In [78]:
final_percent = X.records['level'].sum() / (7*weeks) / (weight * 1000) * 100
tar_percent = activity * perc * 100
total_cost = round((X.records['level'] / 1000 * df['Price [€/kg]']).sum(), 2)
dayly_cost = total_cost / (7 * weeks)
fat = (X.records['level'] * df['Fat']).sum() / X.records['level'].sum() * 100
r = pd.DataFrame({
    'Index': ['Final Percent', 'Targeted Percent', 'Total Cost', 'Dayly Cost', 'Fat'],
    'Column': [final_percent, tar_percent, total_cost, dayly_cost, fat]
}).round(2)
r

,Index,Column
0,Final Percent,2.04
1,Targeted Percent,2.00
2,Total Cost,87.66
3,Dayly Cost,3.13
4,Fat,14.46


In [14]:
%gams_cleanup --closedown